In [ ]:
import pandas as pd
import datetime as DT
import re

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier

import math
from __future__ import division

from itertools import islice

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import nltk
import re
from collections import Counter
import string
import pprint

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# import vaderSentiment.vaderSentiment

In [ ]:
# vaderSentiment.vaderSentiment

In [ ]:
# from textblob import TextBlob

# 1. AlzU Survey Data Section

In [ ]:
ALZU_RESPONSES_FILE_PATH = '../Data_Folder/2018.09.18_AlzU_ANU-DRI + Responses_Numerical.csv'

## Combine first two rows to generate column names

In [ ]:
two_header_rows = pd.read_csv(ALZU_RESPONSES_FILE_PATH, header=None, nrows = 2, delimiter = ",", keep_default_na=False)

transformed_header_row = []

most_recent_col_first_row = None
for column in two_header_rows:
    primary_header = two_header_rows[column][0]
    sub_header = two_header_rows[column][1]
#     print(primary_header + ": " + sub_header)
    if(primary_header != ""):
        most_recent_col_first_row = primary_header
    transformed_header_row.append(most_recent_col_first_row + "_" + sub_header)

## Load Data with column names


        

In [ ]:
alzu_survey_data = pd.read_csv(ALZU_RESPONSES_FILE_PATH, header=None, skiprows = [0,1], delimiter = ",", keep_default_na=False, names=transformed_header_row)

# alzu_survey_data.head()

In [ ]:
alzu_survey_data.shape

In [ ]:
for idx, col in enumerate(alzu_survey_data.columns):
    print(str(idx) + " : " + str(col))

In [ ]:
tech_prompt_idx = 123
tec_prompt_idx = 124

tech_prompt = alzu_survey_data.columns[tech_prompt_idx]
tec_prompt = alzu_survey_data.columns[tec_prompt_idx]

print("Num respondents to memory prompt: "+ str(alzu_survey_data[alzu_survey_data[tech_prompt] != ''].shape[0]))
print("Num respondents to tech prompt: "+ str(alzu_survey_data[alzu_survey_data[tec_prompt] != ''].shape[0]))

Below I am exploring how many users have submitted multiple surveys.  After inspecting many of these users responses, I decide to only keep their most recent survey response, because empirically it seems like that one has the richest data -- the other submissions tend to be minutes before and have no responses, making me think they were having technical difficulties.

In [ ]:
val_counts = alzu_survey_data['uid_Open-Ended Response'].value_counts()

num_users_multiple_submissions = val_counts[val_counts != 1].shape[0]

print("There are " + str(num_users_multiple_submissions) + " users who submitted multiple surveys.")
# print(val_counts)


In [ ]:
alzu_survey_data.drop_duplicates('uid_Open-Ended Response', keep='first', inplace=True)

# 2. CFT Data Section

In [ ]:
CFT_FILE_PATH = '../Data_Folder/2018.09_CFT_all.csv'

In [ ]:
cft_data = pd.read_csv(CFT_FILE_PATH, delimiter = ",", keep_default_na=False)

In [ ]:
# cft_data.head()

In [ ]:
cft_data.shape

Exploration of Dr. Isaacson's Questions about how many times people have taken the CFT for those that have registered between August 22nd and March 7th.

- Do we use "RegDate" in the CFT dataset as the reference date, or do we have to look into the User list for that?
- Do we care about those that registered or do we need a Final Score?

In [ ]:
cft_data['RegDate'] = pd.to_datetime(cft_data['RegDate'])

from datetime import datetime
start_string = 'Aug 22 2017'
end_string = 'Mar 05 2018'
start_date = datetime.strptime(start_string, '%b %d %Y')
end_date = datetime.strptime(end_string, '%b %d %Y')
mask = (cft_data['RegDate'] > start_date) & (cft_data['RegDate'] <= end_date)
# Select the sub-DataFrame:

cft_btwn_dates = cft_data.loc[mask]

print (str(cft_btwn_dates.shape[0]) + " users have registered for the CFT between " + start_string + " and " + end_string)

In [ ]:
cft_btwn_dates.groupby('EmailAddress').count().groupby('UserID').count().ix[:, 2]
# for idx, row in cft_data.groupby('EmailAddress').count().groupby('UserID').count():
#     print idx, row['FirstName']

In [ ]:
cft_data.columns

## CFT Summary Stats

In [ ]:
unique_cft = cft_data.drop_duplicates('EmailAddress', keep='first') # takes the first (oldest) score

print("Number of tests taken: " + str(cft_data.shape[0]))
print("Number of unique users: " + str(unique_cft.shape[0]))

unique_cft.groupby('Gender').count()

In [ ]:
for cft_col in unique_cft.columns:
    print(cft_col)
    

# 3. Email-UserID Section

In [ ]:
ALZU_USERS_FILE_PATH = '../Data_Folder/2018.09_alzu_users.csv'

In [ ]:
user_data = pd.read_csv(ALZU_USERS_FILE_PATH, delimiter = ",", keep_default_na=False)

In [ ]:
# user_data.head()

## Transformations + Data Cleaning

In [ ]:
user_data['Joined'] = pd.to_datetime(user_data['Joined'])
print(user_data.shape)
user_data = user_data.drop_duplicates('Email', keep='last')
print(user_data.shape)



## User Summary Stats

In [ ]:
print("Number of total users: " + str(user_data.shape[0]))
gender_grouping = user_data['Gender'].value_counts()
num_females = gender_grouping['F']
num_males = gender_grouping['M']

print("Number of male users: " + str(num_males))
print("Number of female users: " + str(num_females))
print("Male:Female Ratio ≈ 1:" + str(num_females / num_males))


In [ ]:
date_indexed = user_data.set_index(user_data["Joined"])
date_indexed['count'] = 1
counts = date_indexed['count'].resample('M').sum().dropna()

counts = counts[2:] #get rid of 2 obviously incorrect ones dating way back

import matplotlib.pyplot as plt
cumulative_counts = counts.cumsum()
cumulative_counts.plot()
plt.show()

# JOIN DATASETS

In [ ]:
unique_cft.rename(index=str, columns={"EmailAddress": "Email"}, inplace=True)
alzu_survey_data.rename(index=str, columns={"uid_Open-Ended Response": "User ID"}, inplace=True)

In [ ]:
users_joined_cognitive = user_data.merge(unique_cft, on="Email", how="left")

In [ ]:
# print(users_joined_cognitive.shape)
# users_joined_cognitive.head()

In [ ]:
all_data = users_joined_cognitive.merge(alzu_survey_data, on="User ID", how="left")

In [ ]:
all_data.shape

# 4. Exploration of Combined Dataset

In [ ]:
# all_data.head()

In [ ]:
cols = all_data.columns

In [ ]:
for idx, col in enumerate(cols):
    print(str(idx) + " : " + str(col))

In [ ]:
memory_prompt_idx = 176
tech_prompt_idx = 177

memory_prompt = cols[memory_prompt_idx]
tech_prompt = cols[tech_prompt_idx]

### Peform some General Data Transformations that do not feel model or experiment-specific

Drop all rows in which they have neither Survey Data or CFT data

In [ ]:
survey_and_cft_columns = cols[range(9, 180)]

all_data.dropna(how='all', subset=[survey_and_cft_columns], inplace=True)

Rename gender column and replace with 0 (female) and 1 (male)

In [ ]:
all_data.rename(index=str, columns={"Gender_x": "Gender"}, inplace=True)

all_data['Gender'].replace(['F','M'],[0,1],inplace=True)

Create an Age column based on the DoB column and get some stats on age

In [ ]:
now = pd.Timestamp(DT.datetime.now())


dobs = pd.to_datetime(all_data[all_data['DoB'].notnull()]['DoB'])
dobs = dobs.where(dobs < now, dobs -  np.timedelta64(100, 'Y'))
ages = (now - dobs).astype('<m8[Y]')    # 3

all_data['Age'] = ages


In [ ]:
join_dates = pd.to_datetime(all_data[all_data['Joined'].notnull()]['Joined'])

join_dates = join_dates.where(join_dates < now, join_dates -  np.timedelta64(100, 'Y'))

days_as_member = (now - join_dates).astype('<m8[D]')

all_data['Days_As_Member'] = days_as_member

Replace NANs in the prompt columns with empty strings 

In [ ]:
all_data[memory_prompt].fillna('', inplace=True)
all_data[tech_prompt].fillna('', inplace=True)

Format Scores as Numerics

In [ ]:
all_data['FinalScore'] = pd.to_numeric(all_data['FinalScore'])
all_data['MouseScore'] = pd.to_numeric(all_data['FinalScore'])

### Summary Stats on Transformed "All Data" Dataset

In [ ]:
print(all_data.shape)
print("Average Age:")
print(all_data['Age'].mean())
print("Median Age:")
print(all_data['Age'].median())

gender_grouping = all_data['Gender'].value_counts()
num_females = gender_grouping[0]
num_males = gender_grouping[1]

print("Number of male users: " + str(num_males))
print("Number of female users: " + str(num_females))
print("Male:Female Ratio ≈ 1:" + str(num_females / num_males))

print("Average membership Duration:")
print(all_data['Days_As_Member'].mean())

print(all_data[((all_data[memory_prompt] != "") | (all_data[tech_prompt] != ""))].shape[0])

# Model for users with cognitive scores

## Experiment 1:  Ignore the written portion and identify best ML model for those with Cognitive Scores

### 1.1 Summary Stats on those that have taken Cognitive Tests

In [ ]:
cog_takers = all_data[all_data['FinalScore'].notnull()]
print("Num respondents with final cognitive score: " + str(cog_takers.shape[0]))

print("Average CFT Score: " + str(cog_takers['FinalScore'].mean()))
print("Median CFT Score: " + str(cog_takers['FinalScore'].median()))
print("Standard Deviation of CFT Score: " + str(cog_takers['FinalScore'].std()))

In [ ]:
red_zone = cog_takers[cog_takers['FinalScore'] <= 38]
orange_zone = cog_takers[((cog_takers['FinalScore'] > 38) & (cog_takers['FinalScore'] <= 43))]
green_zone = cog_takers[cog_takers['FinalScore'] > 43]

objects = ('Significant Risk (<38)', 'At Risk (<43, >38)', 'Normal (>43)')
y_pos = np.arange(len(objects))
performance = [red_zone.shape[0],orange_zone.shape[0], green_zone.shape[0]]
 
barlist=plt.bar(y_pos, performance, align='center', alpha=0.5)

barlist[0].set_color('#FF0000')
barlist[1].set_color('#FFBF00')
barlist[2].set_color('g')

plt.xticks(y_pos, objects)
plt.ylabel('Number of Participants')
plt.title('AlzU CFT Performance')
 
plt.show()

print(performance)

### 1.2: Pre-Process 

In [ ]:
X = all_data.copy()

#### For those that have not taken CFT, replace value with mean

In [ ]:
avg_cft_score = cog_takers['FinalScore'].mean()

X['FinalScore'].fillna(avg_cft_score, inplace=True)

#### Pre-process the features

Convert the alzU columns to ints, and Replace "NaN" and empty strings with the mean of column

In [ ]:
idx_of_first_alzu_q = 62
idx_of_last_non_prompt_alz_u_q = 175
open_ended_alzu_indices = [64, 99, 120, 122, 124, 134, 171,175] #columns where they could enter in anything 

for col_idx in range(idx_of_first_alzu_q, idx_of_last_non_prompt_alz_u_q + 1):
    col_name = cols[col_idx]
    print(col_name)
    if (col_idx in open_ended_alzu_indices):
        continue
    print(X[col_name].unique())
    
    # first convert all nan and empty to -1, so that the conversion of the column to astype int doesn't break
    X[col_name].fillna(-1, inplace=True)
    print(X[col_name].unique())
    X[col_name].replace([''], [-1], inplace=True)
    X[col_name].replace(['1.11111E+77'], [-1], inplace=True) # handle the bizarre entry that was breaking this
    X[col_name] = (X[col_name]).astype(int)
    
    X[col_name].replace([-1], [np.NaN], inplace=True)
    column_mean = X[col_name].mean()
    X[col_name].fillna(column_mean, inplace=True)
    
    
    
    print(str(col_idx) + ": " + col_name + ": " +str(X[col_name].dtype))
    print(X[col_name].unique())

Transform some true false columns

In [ ]:
def transform_true_false_col(col_name):
    replaced_NaNs = X[col_name].fillna(-1)
    replaced_t_f = replaced_NaNs.replace([False,True],[0,1])    
    replaced_t_f = replaced_t_f.astype(int)
    replaced_t_f.replace([-1], [np.NaN], inplace=True)
    column_mean = replaced_t_f.mean()
    replaced_t_f.fillna(column_mean, inplace=True)
    return replaced_t_f


X['MemConcern'] = transform_true_false_col('MemConcern')
X['ForgetFriends'] = transform_true_false_col('ForgetFriends')
X['PutThings'] = transform_true_false_col('PutThings')
X['ForgetWords'] = transform_true_false_col('ForgetWords')
X['LoseWay'] = transform_true_false_col('LoseWay')
X['IsMemoryWorse'] = transform_true_false_col('IsMemoryWorse')

Transform some yes no and other string columns...Taking some liberties here with how to encode them

In [ ]:
X['FamilyHistory'].fillna(-1, inplace=True)
X['FamilyHistory'].replace(['', 'no', 'yes'], [-1,0,1], inplace=True)
X['FamilyHistory'].replace([-1], [np.NaN], inplace=True)

X['FamilyHistoryAge'].fillna(-1, inplace=True)
X['FamilyHistoryAge'].replace(['', 'Select', 'none', 'mild', 'other', 'alzh'], [-1,-1,0,1,2,3], inplace=True)
X['FamilyHistoryAge'].replace([-1], [np.NaN], inplace=True)

X['Supplements'].fillna(-1, inplace=True)
X['Supplements'].replace(['Select', 'No', 'Yes'], [-1,0,1], inplace=True)
X['Supplements'].replace([-1], [np.NaN], inplace=True)


Drop columns that we know are not good for ML models

In [ ]:
X = X.drop(['Joined', 'User ID', 'First Name', 'Email', 'Lesson Group', 'UserID', 'FirstName', 'Respondent ID_', 'Collector ID_', 'IP Address_', 'Email Address_', 'First Name', 'Last Name_', 'Gender_y', 'DoB', 'joined_Open-Ended Response', 'email_Open-Ended Response', 'Start Date_', 'End Date_', 'First Name_', 'Custom Data 1_'], axis=1)

Drop columns from the CFT Dataset, THAT MAY BE OF USE LATER, but are just of inconvenient data types...Also we need to confirm that they do not influence the FinalScore

In [ ]:
X = X.drop(['RegDate','DateTaken','Homoscyteine', 'Source', 'Working', 'CompUser', 'DeviceUsed'], axis=1)

In [ ]:
X = X.drop(['recallscore', 'placescore', 'MouseScore', 'Country', 'Postcode', 'Ethnicy', 'Interrupted', 'Functioned', 'FastInternet', 'ScreenClear', 'MaritalStatus', 'Dependents', 'Occupation', 'PrimaryIncome', 'HouseholdIncome', 'PrimaryIncomeOccupation', 'FirstPriority', 'SecondPriority', 'OtherPriority', 'ass', 'combocs'], axis=1)

There are many questions, particularly in the CFT portion, where there are no responses.  We drop those columns below

In [ ]:
for col in X.columns:
    if (X[col].dtype != 'int64' and X[col].dtype != 'float64'):
        if(len(X[col].unique()) == 1 and X[col].unique() == ['']):
            X.drop(col, axis=1, inplace=True)
        


Drop the alzU columns where the format is open-ended.  Some of them have interesting data, but hard to think about how to convert them into numeric at this time.  Come back to this.

In [ ]:
for open_ended_idx in open_ended_alzu_indices:
    col_name = cols[open_ended_idx]
    X.drop(col_name, axis=1, inplace=True)

Drop the written responses field for this round

In [ ]:
X.drop(memory_prompt, axis=1, inplace=True)
X.drop(tech_prompt, axis=1,inplace=True)

In [ ]:
# make sure that all columns are ints or floats and convert NaN into mean for all columns
for col in X.columns:
    if (X[col].dtype !='int64' and X[col].dtype !='float64'):
        print("PROBLEM! COLUMN IS NOT IN NUMERICAL FORMAT")
        print(col + ": " + str(X[col].dtype))
    else:
        column_mean = X[col].mean()
        X[col].fillna(column_mean, inplace=True)

### 1.2 Classification

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

def run_and_print_models(x_train, y_train, x_test, y_test):
    features_importance_dict = {}
    
    #increments a features "importance" based on its weight in a model and the performance of that model
    def update_features_dict(feature_weights, performance):
#         print("FEATURE WEIGHTS")
#         print(feature_weights)       
        absolute_feature_weights = np.absolute(feature_weights)
#         print("ABS VALUE")
#         print(absolute_feature_weights)
        sorted_feature_weights_indices = np.argsort(absolute_feature_weights) # list of feature indices, sorted ascending by absolute value of feature weight 
#         print("Num FEATURES: " + str(len(feature_weights)))
        for ranking, feature_index in enumerate(sorted_feature_weights_indices):            
            feature_name = x_train.columns[feature_index]
#             print("Feature: ")
#             print(feature_name)
#             print("Ranking: ")
#             print(ranking)
            feature_ranking = ranking * performance
#             print("Travis Feature Ranking Rating:")
#             print(feature_ranking)
#             print(feature_ranking)
            if feature_name in features_importance_dict:
                features_importance_dict[feature_name] += feature_ranking
            else:
                features_importance_dict[feature_name] = feature_ranking
    
    def get_top_n_features(n, feature_weights):
        top_n_idx = np.argsort(feature_weights)[-n:] #SHOULD I BE USING ABSOLUTE VALUE?
        top_n_features = [x_train.columns[i] for i in top_n_idx]
        return top_n_features
    
    models = {
        "3-Nearest-Neighbors": KNeighborsClassifier(n_neighbors=3),
        "Linear Regression": LinearRegression(),
        "Decision Tree": tree.DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=2,
                                 random_state=0),
        "Support Vector Machines": svm.SVC(kernel='linear')    
    }
    
    for name, model in models.iteritems():
        model.fit(x_train, y_train)
        performance = model.score(x_test, y_test)
        print(name + " Classification Accuracy: " + str(performance))
        if (name == "3-Nearest-Neighbors"):
            continue
        elif (name == "Support Vector Machines"):
            feature_weights = model.coef_[0]
        elif(name=="Linear Regression"):
            feature_weights = model.coef_
        else:
            feature_weights = model.feature_importances_
#         print(name + " Top Features: ")
#         for idx, feat in enumerate(get_top_n_features(5, feature_weights)):
#             print(str(idx + 1) + ": " + feat)
        
        update_features_dict(feature_weights, performance)
    import operator

    sorted_x = sorted(features_importance_dict.items(), key=operator.itemgetter(1), reverse=True)
    for idx, feat_score_and_weight in enumerate(sorted_x):
        if (idx > 9):
            break
        print(str(idx + 1) + ": " + str(feat_score_and_weight))

### 1.2.1: Two-Classes

#### Classify cog scores on Median (above median = 1, below median = 0)

In [ ]:
# X_two_class = X.copy()
# median_score = X_two_class['FinalScore'].median()

# X_two_class.loc[X_two_class['FinalScore'] < median_score, 'FinalScore'] = 0
# X_two_class.loc[X_two_class['FinalScore'] >= median_score, 'FinalScore'] = 1

# y_two_class = X_two_class['FinalScore']
# X_two_class = X_two_class.drop(['FinalScore'], axis=1)

# X_train_two, X_test_two, y_train_two, y_test_two = train_test_split(X_two_class,y_two_class)


# print(X_train_two.shape)
# print(X_test_two.shape)
# print(y_train_two.shape)
# print(y_test_two.shape)

# print("Label Division...........")
# print(y_two_class[y_two_class == 0].count())
# print(y_two_class[y_two_class == 1].count())


In [ ]:
# run_and_print_models(X_train_two, y_train_two, X_test_two, y_test_two)

### 1.2.2: Three-Class Classification

#### Split based on the scoring categories provided here: http://www.foodforthebrain.org/alzheimers-prevention/take-the-test/interpreting-results.aspx

In [ ]:
# X_three_class = X.copy()

# X_three_class.loc[X_three_class['FinalScore'] <= 38, 'FinalScore'] = 0
# X_three_class.loc[((X_three_class['FinalScore'] > 38) & (X_three_class['FinalScore'] <= 43)), 'FinalScore'] = 1
# X_three_class.loc[X_three_class['FinalScore'] > 43, 'FinalScore'] = 2

# y_three_class = X_three_class['FinalScore']
# X_three_class = X_three_class.drop(['FinalScore'], axis=1)

# X_train_three, X_test_three, y_train_three, y_test_three = train_test_split(X_three_class,y_three_class)

In [ ]:
# run_and_print_models(X_train_three, y_train_three, X_test_three, y_test_three)

## Experiment 2: Analysis of best models for Writing Samples

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.naive_bayes import MultinomialNB
# import nltk
# import re
# from collections import Counter
# import string
# import pprint

# have_writing = all_data[((all_data[memory_prompt] != "") | (all_data[tech_prompt] != ""))].copy()

# print(have_writing.shape)

# writing_samples = have_writing.iloc[:,memory_prompt_idx:tech_prompt_idx + 1]

# print(writing_samples.shape)

# cft_scores = have_writing['FinalScore']

# print(cft_scores.shape)

# #replace NaN with mean
# cft_median_score = cft_scores.median()
# cft_mean_score = cft_scores.mean()

# cft_scores.fillna(cft_mean_score, inplace=True)
# print(cft_median_score)
# print(cft_scores.head())




# print(cft_scores.head())


In [ ]:
# have_writing

### Experiment 2.1: Two-class Classification using TF-IDF features 

In [ ]:
# # https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.where.html#pandas.Series.where
# two_class_cft = cft_scores.copy()
# two_class_cft = two_class_cft.where(two_class_cft >= cft_median_score, 0) 
# two_class_cft = two_class_cft.where(two_class_cft < cft_median_score, 1)

In [ ]:
# ## Models to test
# model_dict = {
#     'Naive Bayes': MultinomialNB(),
#     'Support Vector Machines': SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None),
#     'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=2,
#                                  random_state=0),
#     'Decision Tree': tree.DecisionTreeClassifier(),
#     "3-Nearest-Neighbors": KNeighborsClassifier(n_neighbors=3)
# }

#### Fit Memory Responses Alone

In [ ]:
# mem_prompt = have_writing.iloc[:,memory_prompt_idx]

# print(mem_prompt.shape)
# mem_prompt.head()

In [ ]:
# mem_train_two, mem_test_two, score_train_two, score_test_two = train_test_split(mem_prompt,two_class_cft)

In [ ]:
# best_mem_clf = {
#     'Name': None,
#     'FitModel':None,
#     'Score': 0
# }
# for model_name in model_dict:
#     model = model_dict[model_name]
#     clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', model)])
#     clf.fit(mem_train_two, score_train_two)

#     predicted = clf.predict(mem_test_two)
#     predict_accuracy = np.mean(predicted == score_test_two)
#     if (predict_accuracy >= best_mem_clf['Score']):
#         best_mem_clf['Name'] = model_name
#         best_mem_clf['FitModel'] = clf
#         best_mem_clf['Score'] = predict_accuracy
#     print(model_name + " Prediction Accuracy:")
#     print(predict_accuracy)

In [ ]:
# best_mem_clf

In [ ]:
# for step in best_mem_clf['FitModel'].steps: 
#     if step[0] == 'clf':
#         feature_importances = step[1].feature_importances_ #get_params()

# feature_importances.shape

#### Fit Tech Responses Alone

In [ ]:
# tech_prompt_responses = have_writing.iloc[:,tech_prompt_idx]
# print(tech_prompt_responses.shape)
# print(tech_prompt_responses.head())

# tech_train_two, tech_test_two, score_train_two, score_test_two = train_test_split(tech_prompt_responses,two_class_cft)

In [ ]:
# for model_name in model_dict:
#     model = model_dict[model_name]
#     clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', model)])
#     clf.fit(tech_train_two, score_train_two)

#     predicted = clf.predict(tech_test_two)
#     predict_accuracy = np.mean(predicted == score_test_two) 
#     print(model_name + " Prediction Accuracy:")
#     print(predict_accuracy)

### Experiment 2.2: Three-class Classification using TF-IDF features 

In [ ]:
# three_class_cft = cft_scores.copy()

# three_class_cft = three_class_cft.where(three_class_cft <= 38, 0)
# three_class_cft = three_class_cft.where((three_class_cft > 38 & three_class_cft <=43), 1)
# three_class_cft = three_class_cft.where(three_class_cft > 43, 2)

In [ ]:
# def split_words(sample_string):
#     return re.sub('['+string.punctuation.replace('\'','')+']',' ',sample_string).split()

# # Number of Words in Responses
# print(len(split_words(sample_mem)))
# print(len(split_words(sample_tech)))

In [ ]:
# # Number of Unique Words in Response
# print(len(np.unique(split_words(sample_mem))))
# print(len(np.unique(split_words(sample_tech))))

In [ ]:
# #Honore's lexical richness calculation

# def calc_honores(sample_string):
#     split_string = re.sub('['+string.punctuation.replace('\'','')+']',' ',sample_string).split()
#     N = len(split_string) #number of words
#     uniq_words, counts = np.unique(split_string, return_counts = True)
#     v = len(uniq_words) # number of unique words
#     v1 = len(np.where(counts==1)[0])
    
#     honores = (100*math.log(N))/(1-v1/v)
#     return honores

# print(calc_honores(sample_mem))
# print(calc_honores(sample_tech))

# print(calc_honores("I went to store then I went to the gym then I went to the house"))

In [ ]:
# #Brunet's W index
# def calc_brunets(sample_string):
#     split_string = re.sub('['+string.punctuation.replace('\'','')+']',' ',sample_string).split()
#     N = len(split_string) #number of words
#     uniq_words, counts = np.unique(split_string, return_counts = True)
#     v = len(uniq_words) # number of unique words
#     brunets = N**(v-0.165)
#     return brunets

# print(calc_brunets(sample_mem))
# print(calc_brunets(sample_tech))

## Experiment 3: Use Renee's Correlated Features to Build a Model

Summary Stats

In [ ]:
#https://raw.githubusercontent.com/dwyl/english-words/master/words.txt
with open('words.txt','r') as words:
    word_list = words.read().splitlines()

word_list = [w.lower() for w in word_list]


len(word_list)

In [ ]:
import nltk
nltk.download('tagsets')
nltk.help.upenn_tagset()




def catch(func, handle=lambda e : e, *args, **kwargs):
    try:
        return func(*args, **kwargs)    
    except ZeroDivisionError as z:
        return None
    except Exception as e:
        return handle(e)


# ## Tokenization

# In[69]:


#Mis-spellings, word length
#rm puctuation
#keep contractions, posessives
def list_tokenize(response):
    return re.sub('['+string.punctuation.replace('\'','')+']',' ',response).split()


# ## Information extraction nltk

# In[70]:


#information-extraction
#https://www.nltk.org/book/ch07.html
def ie_preprocess(document):
    #sentence segmentation
    sentences = nltk.sent_tokenize(document) 
    #tokenization
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    #pos tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences] 
    
    return sentences


# ## Uniq, V

# In[71]:


def uniq_words (response,token_list=None, counts_vector_return=False,uniq_set_return=False):
    if token_list is None: token_list = list_tokenize(response)
    if counts_vector_return and uniq_set_return: return np.unique(token_list,return_counts=True)
    
    if counts_vector_return and not uniq_set_return: return np.unique(token_list,return_counts=True)[1]
    
    if uniq_set_return and not counts_vector_return: return np.unique(token_list)
    #typical case, investigating number of uniq responses
    
    return len(np.unique(token_list))
        


# ## V1 

# In[72]:


def single_appearances (response,counts=None):
    if counts is None: counts = uniq_words(response,counts_vector_return=True)
    return len(np.where(counts==1)[0])


# ## N-grams

# In[73]:


#http://locallyoptimal.com/blog/2013/01/20/elegant-n-gram-generation-in-python/
#input
def find_bigrams(response, return_list=False):
    input_list = list_tokenize(response)
    bigrams = zip(input_list, input_list[1:])
    if return_list: return [b for b in bigrams]
    else: return bigrams


# ## Word count

# In[74]:


def word_count(response):
    return len(list_tokenize(response))


# ## Sentence count

# In[75]:


def sentence_count(response):
    return len(ie_preprocess(response))


# ### POS tagging

# In[76]:


#information-extraction
#https://www.nltk.org/book/ch07.html
def ie_preprocess(document):
    #sentence segmentation
    sentences = nltk.sent_tokenize(document) 
    #tokenization
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    #pos tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences] 
    
    return sentences


# ## Rates

# In[77]:


def pos_rate(response,pos_response=None,tag_set=None,denom_set=None,N=None,nn=False,pn=False,vb=False,proper=None):
    
    if pos_response is None: pos_response = ie_preprocess(response)
        
    
    if tag_set is None:
        #common nouns, excluding proper nouns
        if nn: tag_set = ['NN','NNS']
        elif vb: tag_set = ['VB','VBD','VBG','VBN','VBP','VBZ'] 
        elif pn: tag_set = ['PRP','PRP$']
        elif proper: 
            tag_set = ['PRP','PRP$','WP','WP$']
            denom_set = ['PRP','PRP$','WP','WP$','NN','NNP','NNPS','NNS']
            
        
    pos_count = sum([1 for sentence in pos_response for word in sentence if word[1] in tag_set])
    
    if denom_set: 
        denom_count = sum([1 for sentence in pos_response for word in sentence if word[1] in denom_set])
    else:
        if N is None: N = word_count(response)
        denom_count = N
    
    if (denom_count == 0):
        return 0
    else:
        return pos_count/denom_count
        

#         sum([1 for sentence in pos_test for word in sentence if word[1] in ['VB','VBD','VBG','VBN','VBP','VBZ'] ])


# ## Honore's R 
# ![image.png](attachment:image.png)
# https://books.google.com/books?id=CwC4CwAAQBAJ&pg=PA111&lpg=PA111&dq=honore+statistic+lexical+richness&source=bl&ots=LPnM2j9oX5&sig=HGObkoMYRy6lLsgc80Y7tUMT7vk&hl=en&sa=X&ved=2ahUKEwjv0ubE7vTdAhXwYd8KHVRfCiMQ6AEwCXoECAUQAQ#v=onepage&q=honore&f=false
# 
# http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.2.1359&rep=rep1&type=pdf
# 
# http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.484.4648&rep=rep1&type=pdf
# 
# viii) Honoré’s R statistic.
# If V1 denotes the number of hapax legomena recorded, then Honoré (1979) defined his
# statistic as:
# 
# *R = 100log(N)/(1 - V1/V)*
# 
# As a measure of vocabulary richness which has the virtue of being insensitive to text
# length, it has been used in stylometric studies by Holmes (1992) and Holmes and
# Stylometric analysis of conversational speech
# Forsyth (1995). Values of R typically range from 1000 to 2000 with higher values
# implying richer vocabularies in the sense that a greater number of words appear
# infrequently.
# 
# 
# https://pdfs.semanticscholar.org/11f9/ef33ad001f7638ba29ee8109077de92eb1bb.pdf
# 
# 
# Assuming log == base e...fits with 1000-2000 assumption
# 
# 
# 

# In[78]:


def honore_r(response,v=None,v1=None, N=None):
    if N is None: N = word_count(response)
    if v is None: v = uniq_responses(response)
    if v1 is None: 
        counts = uniq_responses(response,counts_vector_return=True)
        v1 = len(np.where(counts==1)[0])
    return (100*math.log(N))/(1-v1/v)


# ## Brunet's W
# W = N**(V −0.165)
# 
# (vii) Brunet’s W index.
# This index, devised by Brunet (1978) and used successfully by Holmes and Forsyth
# (1995), is a measure of vocabulary richness which is insensitive to text length. It is
# defined as:
# 
# W= N^(V-0.165)
# where N is the text length, V the number of different words and (-0.165) is a scaling
# constant proposed by Brunet. The measure generally varies between 10 and 20 with a
# low value indicating a lexically richer speech.
# https://pdfs.semanticscholar.org/11f9/ef33ad001f7638ba29ee8109077de92eb1bb.pdf

# In[79]:


def brunet_w(response,N=None, v=None):
    if N is None: N = word_count(response)
    if v is None: v = uniq_responses(response)
    return N**(v-0.165)


# ## Moving window

# In[80]:


#10-window size generator
#https://docs.python.org/release/2.3.5/lib/itertools-example.html
def window(seq, n=10):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def coord_conjunctions_init(response):
    return sum([1 for sentence in ie_preprocess(response) if sentence[0][1] == 'CC'])

def misspelt(response):
    return sum([1 for word in list_tokenize(response) if word.lower() not in word_list])

def calc_MATTR(mr):
    window_iterable = window(list_tokenize(mr),n=min(len(list_tokenize(mr)),10))
    return np.mean([len(np.unique(mw))/min(len(list_tokenize(mr)),10) for mw in window_iterable])

In [ ]:
# # http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# vader = SentimentIntensityAnalyzer()

### Exp 3.1: See how language features compare to non-lang for users with memory responses

In [ ]:
have_mem_indices = all_data.index[all_data[memory_prompt] != ""] #Gets the indices of rows that have memory response
print(have_mem_indices.shape)


In [ ]:
have_mem_indices

In [ ]:
mem_cft_scores = pd.Series(all_data.loc[have_mem_indices]['FinalScore'], index=have_mem_indices) #Will be used as target for both language and non-language experiments
print(mem_cft_scores.shape)

#replace NaN with mean
mem_cft_median_score = mem_cft_scores.median()
mem_cft_mean_score = mem_cft_scores.mean()
mem_cft_scores.fillna(mem_cft_mean_score, inplace=True)


### MAKE A THREE-CLASS TARGET, BASED ON CFT WEBSITE
mem_THREE_class_cft = mem_cft_scores.copy()

mem_THREE_class_cft = mem_THREE_class_cft.where(mem_THREE_class_cft > 38, 0)
mem_THREE_class_cft = mem_THREE_class_cft.where(mem_THREE_class_cft < 43, 2)
mem_THREE_class_cft = mem_THREE_class_cft.where(((mem_THREE_class_cft == 0) | (mem_THREE_class_cft == 2)), 1)

### Hold Out Test Set

In [ ]:
held_out_test_mem_idx = np.random.choice(have_mem_indices,(1,int(.20*len(have_mem_indices))),replace=False)[0]

array([['40712', '44816', '43211', '51913', '41880', '39575', '29112',
        '54397', '29004', '52080', '44345', '57361', '29113', '49691',
        '40020', '38603', '46323', '29136', '55057', '54236', '43644',
        '47950', '29670', '55991', '41751', '30279', '35853', '58820',
        '29615', '35337', '59682', '42780', '40118', '54884', '29056',
        '32304', '51163', '43205', '32565', '29234', '53994', '58441',
        '29084', '32601', '34097', '46015', '42587', '36881', '47461',
        '49925', '39966', '54091', '43429', '29608', '43700', '45405',
        '29563', '54740', '30384', '29768', '32182', '31723', '32871',
        '50433', '40693', '39695', '46613', '41045', '40854', '38440',
        '38649', '45884']], dtype=object)

In [ ]:
# held_out_test_mem_idx = ['40712', '44816', '43211', '51913', '41880', '39575', '29112', '54397', '29004', '52080', '44345', '57361', '29113', '49691', '40020', '38603', '46323', '29136', '55057', '54236', '43644', '47950', '29670', '55991', '41751', '30279', '35853', '58820', '29615', '35337', '59682', '42780', '40118', '54884', '29056', '32304', '51163', '43205', '32565', '29234', '53994', '58441', '29084', '32601', '34097', '46015', '42587', '36881', '47461', '49925', '39966', '54091', '43429', '29608', '43700', '45405', '29563', '54740', '30384', '29768', '32182', '31723', '32871', '50433', '40693', '39695', '46613', '41045', '40854', '38440', '38649', '45884']

In [ ]:
y_mem_heldout = mem_THREE_class_cft[held_out_test_mem_idx]

In [ ]:
y_mem_heldout.shape

### Dev Set 

In [ ]:
train_dev_mem_ix = [m for m in have_mem_indices if m not in held_out_test_mem_idx]

In [ ]:
dev_mem_idx = np.random.choice(train_dev_mem_ix,(1,int(.20*len(train_dev_mem_ix))),replace=False)[0]

In [ ]:
# dev_mem_idx = ['46047', '58728', '42364', '39539', '39862', '30087', '58354',
#         '47480', '56239', '46765', '39880', '34753', '31795', '45647',
#         '43233', '32846', '55823', '31075', '43830', '39573', '53678',
#         '57834', '32144', '39171', '52174', '39582', '33976', '45821',
#         '43155', '53050', '52053', '53125', '39082', '59103', '37078',
#         '33444', '29051', '40017', '39830', '42876', '29970', '31871',
#         '31482', '39319', '54924', '44439', '38414', '29502', '30865',
#         '32233', '43247', '57709', '53472', '40938', '29129', '57407',
#         '42040', '40253']

In [ ]:
y_mem_dev = mem_THREE_class_cft[dev_mem_idx]

In [ ]:
y_mem_dev.shape

### Train Set 

In [ ]:
train_mem_idx =  [i for i in train_dev_mem_ix if i not in dev_mem_idx]

In [ ]:
len(train_mem_idx)

In [ ]:
y_mem_train = mem_THREE_class_cft[train_mem_idx]

In [ ]:
from sklearn.metrics import classification_report


def prediction_accuracies(X_train, X_test, y_train, y_test):
    ## Models to test
    model_dict = {
        'Naive Bayes': MultinomialNB(),
        'Support Vector Machines': SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None),
        'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=2,
                                     random_state=0),
        'Decision Tree': tree.DecisionTreeClassifier(),
        "3-Nearest-Neighbors": KNeighborsClassifier(n_neighbors=3)
    }

    for model_name in model_dict:
        model = model_dict[model_name]
        model.fit(X_train, y_train)

        predicted = model.predict(X_test)
        predict_accuracy = np.mean(predicted == y_test)
        
        if (model_name == 'Naive Bayes'):
            nb_predic_acc = predict_accuracy
        if (model_name == 'Support Vector Machines'):
            svm_predic_acc = predict_accuracy
        if(model_name == 'Random Forest'):
            rf_predic_acc = predict_accuracy
        if(model_name == 'Decision Tree'):
            dec_acc = predict_accuracy
        if(model_name == "3-Nearest-Neighbors"):
            knn_acc = predict_accuracy
        print(model_name + " Prediction Accuracy:")
        print(predict_accuracy)
        
        target_names = ['Significant Risk', 'At Risk', 'Normal']
        
        print(classification_report(y_test, predicted, target_names=target_names))
    return nb_predic_acc, svm_predic_acc, rf_predic_acc, dec_acc, knn_acc

In [ ]:
# def prediction_accuracy(model_name, X_train, X_test, y_train, y_test):
#     model_dict = {
#         'Naive Bayes': MultinomialNB(),
#         'Support Vector Machines': SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None),
#         'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=2,
#                                      random_state=0),
#         'Decision Tree': tree.DecisionTreeClassifier(),
#         "3-Nearest-Neighbors": KNeighborsClassifier(n_neighbors=3)
#     }

#     model = model_dict[model_name]
#     model.fit(X_train, y_train)

#     predicted = model.predict(X_test)
#     predict_accuracy = np.mean(predicted == y_test)

#     return predict_accuracy

#### Exp 3.1, Part 1: Run models on Non-Language Data 

In [ ]:
mem_responses_no_text = X.loc[have_mem_indices]
mem_responses_no_text.drop('FinalScore', axis=1, inplace=True) #Those with memory respones: all data besides text

In [ ]:
no_text_mem_train = mem_responses_no_text.loc[train_mem_idx]

no_text_mem_dev = mem_responses_no_text.loc[dev_mem_idx]

nb_no_text_mem_acc, svm_no_text_mem_acc, rf_no_text_mem_acc, dec_tree_no_text_mem_acc, knn_no_text_mem_acc = prediction_accuracies( no_text_mem_train, no_text_mem_dev, y_mem_train, y_mem_dev)


In [ ]:
no_text_mem_train.shape

In [ ]:
for no_text_mem_col in no_text_mem_train.columns:
    dropped_X_train = no_text_mem_train.drop(no_text_mem_col, axis=1)
    dropped_X_dev = no_text_mem_dev.drop(no_text_mem_col, axis=1)
    
    model = RandomForestClassifier(n_estimators=100, max_depth=2,
                                     random_state=0)
    model.fit(dropped_X_train, y_mem_train)

    predicted = model.predict(dropped_X_dev)
    predict_accuracy = np.mean(predicted == y_mem_dev)
    
    if(predict_accuracy < rf_no_text_mem_acc):
        print(no_text_mem_col)
        print(predict_accuracy)
        print("..........................................")

#### Exp 3.2, Part 2: Run Models on Language Data

In [ ]:
mem_responses = all_data.loc[have_mem_indices].iloc[:,memory_prompt_idx] #Just responses themselves
print(mem_responses.shape)

memory_response_df = mem_responses.to_frame(name='Response')
memory_response_df['NumWords'] = memory_response_df.apply(lambda row: word_count(row['Response']), axis=1)
memory_response_df['NumSentences'] = memory_response_df.apply(lambda row: sentence_count(row['Response'].decode('utf-8')), axis=1)
memory_response_df['V'] = memory_response_df.apply(lambda row: uniq_words(row['Response']), axis=1)
memory_response_df['V1'] = memory_response_df.apply(lambda row: single_appearances(row['Response']), axis=1)
memory_response_df['MATTR'] = memory_response_df.apply(lambda row: calc_MATTR(row['Response']), axis=1)
memory_response_df['CoordinatingConjunctions'] = memory_response_df.apply(lambda row: coord_conjunctions_init(row['Response'].decode('utf-8')), axis=1)
memory_response_df['Misspellings'] = memory_response_df.apply(lambda row: misspelt(row['Response'].decode('utf-8')), axis=1)
memory_response_df['NounRate'] = memory_response_df.apply(lambda row: pos_rate(row['Response'].decode('utf-8'),nn=True), axis=1)

memory_response_df['VerbRate'] = memory_response_df.apply(lambda row: pos_rate(row['Response'].decode('utf-8'),vb=True), axis=1)

memory_response_df['PronounRate'] = memory_response_df.apply(lambda row: pos_rate(row['Response'].decode('utf-8'),pn=True), axis=1) 

memory_response_df['ProperNounRate'] = memory_response_df.apply(lambda row: pos_rate(row['Response'].decode('utf-8'),proper=True), axis=1) 

memory_response_df.drop('Response', axis=1, inplace=True) #get rid of actual text

In [ ]:
memory_response_df.head()

In [ ]:
text_mem_train = memory_response_df.loc[train_mem_idx]

text_mem_dev = memory_response_df.loc[dev_mem_idx]

nb_text_mem_acc, svm_text_mem_acc, rf_text_mem_acc, dec_tree_text_mem_acc, knn_text_mem_acc = prediction_accuracies( text_mem_train, text_mem_dev, y_mem_train, y_mem_dev)

#### Exp 3.2, Part 3:  Combine Text and No-Text and Test Same Models

In [ ]:
combined_mem = mem_responses_no_text.join(memory_response_df)

In [ ]:
combined_mem_train = combined_mem.loc[train_mem_idx]
combined_mem_dev = combined_mem.loc[dev_mem_idx]

In [ ]:
#combined_mem_train, combined_mem_test, combined_score_train, combined_score_test = train_test_split(combined_mem,mem_THREE_class_cft)

In [ ]:
nb_all_mem_acc, svm_all_mem_acc, rf_all_mem_acc, dec_all_text_mem_acc, knn_all_mem_acc = prediction_accuracies( combined_mem_train, combined_mem_dev, y_mem_train, y_mem_dev)

#### Exp 3.2, Part 4:  Add Text Features one at a time

For this experiment, we look only at the effect on Random Forest, which showed an improved prediction accuracy for JUST Lang data and Lang + Non-Lang data   

In [ ]:
for idx, mem_resp_col in enumerate(memory_response_df.columns):
    feature_column = memory_response_df[mem_resp_col]
    concatenated = mem_responses_no_text.join(feature_column)

    X_train = concatenated.loc[train_mem_idx]
    X_dev = concatenated.loc[dev_mem_idx]
   
    model = RandomForestClassifier(n_estimators=100, max_depth=5,
                                     random_state=0)
    model.fit(X_train, y_mem_train)

    predicted = model.predict(X_dev)
    predict_accuracy = np.mean(predicted == y_mem_dev)
    
    if (predict_accuracy > rf_no_text_mem_acc):
        print("Feature Tested: " + mem_resp_col)
        print(predict_accuracy)
        print("................................")
    

Now we take the best individually-performing feature additions and combine them with non-text:

In [ ]:
good_mem_text_features = ['NumSentences', 'V', 'MATTR', 'Misspellings', 'VerbRate']

combined_good = mem_responses_no_text.join(memory_response_df[good_mem_text_features])

X_train = concatenated.loc[train_mem_idx]
X_dev = concatenated.loc[dev_mem_idx]

model = RandomForestClassifier(n_estimators=100, max_depth=5,
                                 random_state=0)
model.fit(X_train, y_mem_train)

predicted = model.predict(X_dev)
predict_accuracy = np.mean(predicted == y_mem_dev)

print(predict_accuracy)

### Exp 4: Do the same with Tech Responses

In [ ]:
have_tech_indices = all_data.index[all_data[tech_prompt] != ""] #Gets the indices of rows that have tech response
print(have_tech_indices.shape)

In [ ]:
tech_cft_scores = pd.Series(all_data.loc[have_tech_indices]['FinalScore'], index=have_tech_indices) #Will be used as target for both language and non-language experiments
print(tech_cft_scores.shape)

#replace NaN with mean
tech_cft_median_score = tech_cft_scores.median()
tech_cft_mean_score = tech_cft_scores.mean()
tech_cft_scores.fillna(tech_cft_mean_score, inplace=True)


### MAKE A THREE-CLASS TARGET, BASED ON CFT WEBSITE
tech_THREE_class_cft = tech_cft_scores.copy()

tech_THREE_class_cft = tech_THREE_class_cft.where(tech_THREE_class_cft > 38, 0)
tech_THREE_class_cft = tech_THREE_class_cft.where(tech_THREE_class_cft < 43, 2)
tech_THREE_class_cft = tech_THREE_class_cft.where(((tech_THREE_class_cft == 0) | (tech_THREE_class_cft == 2)), 1)

### Hold Out Test Set

In [ ]:
held_out_test_tech_idx = np.random.choice(have_tech_indices,(1,int(.20*len(have_tech_indices))),replace=False)[0]

In [ ]:
# held_out_test_tech_idx = ['38874', '44404', '29421', '38698', '40693', '49414', '41751',
#         '56026', '41612', '51136', '38611', '56084', '45787', '56784',
#         '39171', '53312', '47004', '56291', '49527', '38604', '40084',
#         '31837', '45294', '55091', '54529', '59200', '28867', '50329',
#         '29733', '38820', '35401', '28926', '42044', '40020', '55165',
#         '54881', '39399', '48595', '54884', '48604', '57444', '55417',
#         '31120', '43247', '43004', '39012', '31795', '30801', '29079',
#         '39582', '58441', '53994', '46323', '55642', '40227', '49925',
#         '30279', '39452', '43624', '45993', '31871', '43507', '39221',
#         '29088', '41000', '53125', '59703', '38700', '40189', '43429',
#         '32565', '57407', '44151', '59243', '39880', '29258', '45700',
#         '37300', '41703', '31008', '30762', '28932', '39942', '35914']

In [ ]:
y_tech_heldout = tech_THREE_class_cft[held_out_test_tech_idx]

In [ ]:
y_tech_heldout.shape

### Dev Set 

In [ ]:
train_dev_tech_ix = [m for m in have_tech_indices if m not in held_out_test_tech_idx]

In [ ]:
dev_tech_idx = np.random.choice(train_dev_tech_ix,(1,int(.20*len(train_dev_tech_ix))),replace=False)[0]

In [ ]:
# dev_tech_idx = ['40854', '44234', '51913', '41997', '29059', '45772', '40595',
#         '52023', '48606', '38440', '51713', '57709', '39539', '46170',
#         '30642', '32846', '31945', '43248', '34753', '57299', '46047',
#         '51163', '30522', '38686', '43155', '34515', '38754', '44730',
#         '29073', '40732', '60026', '41626', '59232', '44439', '46055',
#         '38603', '30865', '38492', '29090', '38830', '39830', '45647',
#         '57812', '43830', '42443', '56818', '39901', '52767', '38944',
#         '46166', '29333', '54101', '29886', '55823', '48498', '32233',
#         '56239', '44235', '38803', '57834', '29805', '47438', '40530',
#         '56110', '38538', '54924', '45821', '53050']

In [ ]:
y_tech_dev = tech_THREE_class_cft[dev_tech_idx]

In [ ]:
y_tech_dev.shape

### Train Set 

In [ ]:
train_tech_idx =  [i for i in train_dev_tech_ix if i not in dev_tech_idx]

In [ ]:
y_tech_train = tech_THREE_class_cft[train_tech_idx]

#### Exp 4.1, Part 1: Run models on Non-Language Data 

In [ ]:
tech_responses_no_text = X.loc[have_tech_indices]
tech_responses_no_text.drop('FinalScore', axis=1, inplace=True) #Those with tech respones: all data besides text

In [ ]:
no_text_tech_train = tech_responses_no_text.loc[train_tech_idx]

no_text_tech_dev = tech_responses_no_text.loc[dev_tech_idx]

nb_no_text_tech_acc, svm_no_text_tech_acc, rf_no_text_tech_acc, dec_tree_no_text_tech_acc, knn_no_text_tech_acc = prediction_accuracies( no_text_tech_train, no_text_tech_dev, y_tech_train, y_tech_dev)


In [ ]:
#### Exp 4.1, Part 2: Run models on Language Data 

In [ ]:
tech_responses = all_data.loc[have_tech_indices].iloc[:,tech_prompt_idx] #Just responses themselves
print(tech_responses.shape)

tech_response_df = tech_responses.to_frame(name='Response')
tech_response_df['NumWords'] = tech_response_df.apply(lambda row: word_count(row['Response']), axis=1)
tech_response_df['NumSentences'] = tech_response_df.apply(lambda row: sentence_count(row['Response'].decode('utf-8')), axis=1)
tech_response_df['V'] = tech_response_df.apply(lambda row: uniq_words(row['Response']), axis=1)
tech_response_df['V1'] = tech_response_df.apply(lambda row: single_appearances(row['Response']), axis=1)
tech_response_df['MATTR'] = tech_response_df.apply(lambda row: calc_MATTR(row['Response']), axis=1)
tech_response_df['CoordinatingConjunctions'] = tech_response_df.apply(lambda row: coord_conjunctions_init(row['Response'].decode('utf-8')), axis=1)
tech_response_df['Misspellings'] = tech_response_df.apply(lambda row: misspelt(row['Response'].decode('utf-8')), axis=1)
tech_response_df['NounRate'] = tech_response_df.apply(lambda row: pos_rate(row['Response'].decode('utf-8'),nn=True), axis=1)

tech_response_df['VerbRate'] = tech_response_df.apply(lambda row: pos_rate(row['Response'].decode('utf-8'),vb=True), axis=1)

tech_response_df['PronounRate'] = tech_response_df.apply(lambda row: pos_rate(row['Response'].decode('utf-8'),pn=True), axis=1) 

tech_response_df['ProperNounRate'] = tech_response_df.apply(lambda row: pos_rate(row['Response'].decode('utf-8'),proper=True), axis=1) 

tech_response_df.drop('Response', axis=1, inplace=True) #get rid of actual text

In [ ]:
text_tech_train = tech_response_df.loc[train_tech_idx]

text_tech_dev = tech_response_df.loc[dev_tech_idx]

nb_text_tech_acc, svm_text_tech_acc, rf_text_tech_acc, dec_tree_text_tech_acc, knn_text_tech_acc = prediction_accuracies( text_tech_train, text_tech_dev, y_tech_train, y_tech_dev)

#### Exp 4.2, Part 3:  Combine Text and No-Text and Test Same Models

In [ ]:
combined_tech = tech_responses_no_text.join(tech_response_df)

In [ ]:
combined_tech_train = combined_tech.loc[train_tech_idx]
combined_tech_dev = combined_tech.loc[dev_tech_idx]

In [ ]:
nb_all_tech_acc, svm_all_tech_acc, rf_all_tech_acc, dec_all_text_tech_acc, knn_all_tech_acc = prediction_accuracies( combined_tech_train, combined_tech_dev, y_tech_train, y_tech_dev)

#### Exp 4.2, Part 4:  Add Text Features one at a time

For this experiment, we look only at the effect on Random Forest, which showed an improved prediction accuracy for JUST Lang data and Lang + Non-Lang data   

In [ ]:
for idx, tech_resp_col in enumerate(tech_response_df.columns):
    feature_column = tech_response_df[tech_resp_col]
    concatenated = tech_responses_no_text.join(feature_column)

    X_train = concatenated.loc[train_tech_idx]
    X_dev = concatenated.loc[dev_tech_idx]
   
    model = RandomForestClassifier(n_estimators=100, max_depth=5,
                                     random_state=0)
    model.fit(X_train, y_tech_train)

    predicted = model.predict(X_dev)
    predict_accuracy = np.mean(predicted == y_tech_dev)
    
    #if (predict_accuracy > rf_no_text_tech_acc):
    print("Feature Tested: " + tech_resp_col)
    print(predict_accuracy)
    print("................................")
    

# FINAL EXPERIMENTS USING HELD-OUT TEST

### Memory, no Language

In [ ]:
no_text_mem_train = mem_responses_no_text.loc[train_mem_idx]

no_text_mem_test = mem_responses_no_text.loc[held_out_test_mem_idx]

final_nb_no_text_mem_acc, final_svm_no_text_mem_acc, final_rf_no_text_mem_acc, final_dec_tree_no_text_mem_acc, final_knn_no_text_mem_acc = prediction_accuracies( no_text_mem_train, no_text_mem_test, y_mem_train, y_mem_heldout)

In [ ]:
#having trouble getting this from the classification report so writing this down from printed results for now
mem_no_text_precision = (20, 0, 0, 12, 0) #nb,svm, rf, dt,knn
mem_no_text_recall = (22, 0, 0, 11, 0)
mem_no_text_f1 = (21, 0, 0, 12, 0)


# mem_no_text_knn_precision, mem_no_text_knn_recall, mem_no_text_knn_f1 = 51, 60, 55
# mem_no_text_svm_precision, mem_no_text_svm_recall, mem_no_text_svm_f1 = 46,67,54
# mem_no_text_nb_precision, mem_no_text_nb_recall, mem_no_text_knn_f1

In [ ]:
for no_text_mem_col in no_text_mem_train.columns:
    dropped_X_train = no_text_mem_train.drop(no_text_mem_col, axis=1)
    dropped_X_test = no_text_mem_test.drop(no_text_mem_col, axis=1)
    
    model = RandomForestClassifier(n_estimators=100, max_depth=2,
                                     random_state=0)
    model.fit(dropped_X_train, y_mem_train)

    predicted = model.predict(dropped_X_test)
    predict_accuracy = np.mean(predicted == y_mem_heldout)
    
    if(predict_accuracy < final_rf_no_text_mem_acc):
        print(no_text_mem_col)
        print(predict_accuracy)
        print("..........................................")

### Memory, Language

In [ ]:
text_mem_train = memory_response_df.loc[train_mem_idx]

text_mem_test = memory_response_df.loc[held_out_test_mem_idx]

final_nb_text_mem_acc, final_svm_text_mem_acc, final_rf_text_mem_acc, final_dec_tree_text_mem_acc, final_knn_text_mem_acc = prediction_accuracies( text_mem_train, text_mem_test, y_mem_train, y_mem_heldout)

In [ ]:
mem_text_precision = (0, 0, 0, 13, 10)
mem_text_recall = (0, 0, 0, 22, 11)
mem_text_f1 = (0, 0, 0, 17, 11)


In [ ]:
for text_mem_col in text_mem_train.columns:
    dropped_X_train = text_mem_train.drop(text_mem_col, axis=1)
    dropped_X_test = text_mem_test.drop(text_mem_col, axis=1)
    
    model = RandomForestClassifier(n_estimators=100, max_depth=2,
                                     random_state=0)
    model.fit(dropped_X_train, y_mem_train)

    predicted = model.predict(dropped_X_test)
    predict_accuracy = np.mean(predicted == y_mem_heldout)
    
    if(predict_accuracy < final_rf_text_mem_acc):
        print(text_mem_col)
        print(predict_accuracy)
        print("..........................................")

### Memory, Combined

In [ ]:
combined_mem = mem_responses_no_text.join(memory_response_df)

combined_mem_train = combined_mem.loc[train_mem_idx]
combined_mem_test = combined_mem.loc[held_out_test_mem_idx]

final_nb_all_mem_acc, final_svm_all_mem_acc, final_rf_all_mem_acc, final_dec_all_text_mem_acc, final_knn_all_mem_acc = prediction_accuracies( combined_mem_train, combined_mem_test, y_mem_train, y_mem_heldout)

In [ ]:
mem_combined_precision = (18, 0, 0, 17, 0)
mem_combined_recall = (22, 0, 0, 24, 0)
mem_combined_f1 = (20, 0, 0, 13, 0)

In [ ]:
# for idx, mem_resp_col in enumerate(combined_mem.columns):
#     feature_column = combined_mem[mem_resp_col]
#     concatenated = mem_responses_no_text.join(feature_column)

#     X_train = concatenated.loc[train_mem_idx]
#     X_test = concatenated.loc[held_out_test_mem_idx]
   
#     model = tree.DecisionTreeClassifier()
#     model.fit(X_train, y_mem_train)

#     predicted = model.predict(X_test)
#     predict_accuracy = np.mean(predicted == y_mem_heldout)
    
#     if (predict_accuracy > final_rf_all_mem_acc):
#         print("Feature Tested: " + mem_resp_col)
#         print(predict_accuracy)
#         print("................................")

### Tech, No Language

In [ ]:
no_text_tech_train = tech_responses_no_text.loc[train_tech_idx]

no_text_tech_test = tech_responses_no_text.loc[held_out_test_tech_idx]

final_nb_no_text_tech_acc, final_svm_no_text_tech_acc, final_rf_no_text_tech_acc, final_dec_tree_no_text_tech_acc, final_knn_no_text_tech_acc = prediction_accuracies( no_text_tech_train, no_text_tech_test, y_tech_train, y_tech_heldout)

In [ ]:
tech_no_text_precision = (18, 0, 0, 60, 20)
tech_no_text_recall = (22, 0, 0, 33, 11)
tech_no_text_f1 = (20, 4, 0, 0, 43)

In [ ]:
for no_text_tech_col in no_text_tech_train.columns:
    dropped_X_train = no_text_tech_train.drop(no_text_tech_col, axis=1)
    dropped_X_test = no_text_tech_test.drop(no_text_tech_col, axis=1)
    
    model = RandomForestClassifier(n_estimators=100, max_depth=2,
                                     random_state=0)
    model.fit(dropped_X_train, y_tech_train)

    predicted = model.predict(dropped_X_test)
    predict_accuracy = np.mean(predicted == y_tech_heldout)
    
    if(predict_accuracy < final_rf_no_text_tech_acc):
        print(no_text_tech_col)
        print(predict_accuracy)
        print("..........................................")

### Tech, Language

In [ ]:
text_tech_train = tech_response_df.loc[train_tech_idx]

text_tech_test = tech_response_df.loc[held_out_test_tech_idx]

final_nb_text_tech_acc, final_svm_text_tech_acc, final_rf_text_tech_acc, final_dec_tree_text_tech_acc, final_knn_text_tech_acc = prediction_accuracies(text_tech_train, text_tech_test, y_tech_train, y_tech_heldout)

In [ ]:
tech_text_precision = (0, 0, 0, 8, 20)
tech_text_recall = (0, 0, 0, 11, 11)
tech_text_f1 = (0,0,0,10,14)

## Tech, Combined

In [ ]:
combined_tech = tech_responses_no_text.join(tech_response_df)

combined_tech_train = combined_tech.loc[train_tech_idx]
combined_tech_test = combined_tech.loc[held_out_test_tech_idx]

final_nb_all_tech_acc, final_svm_all_tech_acc, final_rf_all_tech_acc, final_dec_all_text_tech_acc, final_knn_all_tech_acc = prediction_accuracies( combined_tech_train, combined_tech_test, y_tech_train, y_tech_heldout)

In [ ]:
tech_combined_precision = (25, 29, 0, 64, 20)
tech_combined_recall = (22, 22, 0, 44, 11)
tech_combined_f1 = (24, 25, 0, 44, 14)

# RESULTS FIGURES SECTION

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


N = 3

act_sig_risk = mem_THREE_class_cft[mem_THREE_class_cft == 0].shape[0] + tech_THREE_class_cft[tech_THREE_class_cft == 0].shape[0]
act_at_risk = mem_THREE_class_cft[mem_THREE_class_cft == 1].shape[0] + tech_THREE_class_cft[tech_THREE_class_cft == 1].shape[0]
act_normal = mem_THREE_class_cft[mem_THREE_class_cft == 2].shape[0] + tech_THREE_class_cft[tech_THREE_class_cft == 2].shape[0]

total_cft = mem_THREE_class_cft.shape[0] + tech_THREE_class_cft.shape[0]

objects = ('Significant Risk (<38)', 'At Risk (<43, >38)', 'Normal (>43)')

mem_CFT = ((act_sig_risk/total_cft)*100, (act_at_risk/total_cft)*100, (act_normal/total_cft)*100)


fig, ax = plt.subplots()

ind = np.arange(N)    # the x locations for the groups
width = 0.35         # the width of the bars
p1 = ax.bar(ind, mem_CFT, width, color='r')


expected_CFT = (6.7, 9.2, 84.1)
p2 = ax.bar(ind + width, expected_CFT, width,
            color='y')

ax.set_title('Expected CFT Score Distribution and Actual Distribution')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('Significant Risk (<38)', 'At Risk (38-43)', 'Normal (>43)'))

ax.legend((p1[0], p2[0]), ('CFT Distribution in Dataset', 'Expected CFT Distribution'))
ax.autoscale_view()

plt.ylabel('% of Total')
plt.show()

In [ ]:
Num_Models = 5

mem_no_text = (final_nb_no_text_mem_acc*100, final_svm_no_text_mem_acc*100, final_rf_no_text_mem_acc*100, final_dec_tree_no_text_mem_acc*100, final_knn_no_text_mem_acc*100)

fig, ax = plt.subplots()

ind = np.arange(Num_Models)    # the x locations for the groups
width = 0.25         # the width of the bars
p1 = ax.bar(ind, mem_no_text, width, color='b')

mem_text = (final_nb_text_mem_acc*100, final_svm_text_mem_acc*100, final_rf_text_mem_acc*100, final_dec_tree_text_mem_acc*100, final_knn_text_mem_acc*100)
p2 = ax.bar(ind + width, mem_text, width,
            color='g')

mem_combined = (final_nb_all_mem_acc*100, final_svm_all_mem_acc*100, final_rf_all_mem_acc*100, final_dec_all_text_mem_acc*100, final_knn_all_mem_acc*100)
p3 = ax.bar(ind + width + width, mem_combined, width,
            color='#00FFFF')

ax.set_title('Model Accuracy on "Memory Users"')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('NB', 'SVM', 'RF', 'DecTree', 'kNN'))

ax.legend((p1[0], p2[0], p3[0]), ('Non-Text Data', 'Text Data', 'Combined'), prop={'size': 6})
ax.autoscale_view()
ax.set_ylim(0,100)
plt.ylabel('Prediction Accuracy (%)')
plt.show()

In [ ]:
Num_Models = 5

tech_no_text = (final_nb_no_text_tech_acc*100, final_svm_no_text_tech_acc*100, final_rf_no_text_tech_acc*100, final_dec_tree_no_text_tech_acc*100, final_knn_no_text_tech_acc*100)

fig, ax = plt.subplots()

ind = np.arange(Num_Models)    # the x locations for the groups
width = 0.25         # the width of the bars
p1 = ax.bar(ind, tech_no_text, width, color='b')

tech_text = (final_nb_text_tech_acc*100, final_svm_text_tech_acc*100, final_rf_text_tech_acc*100, final_dec_tree_text_tech_acc*100, final_knn_text_tech_acc*100)
p2 = ax.bar(ind + width, tech_text, width,
            color='g')


tech_combined = (final_nb_all_tech_acc*100, final_svm_all_tech_acc*100, final_rf_all_tech_acc*100, final_dec_all_text_tech_acc*100, final_knn_all_tech_acc*100)
p3 = ax.bar(ind + width + width, tech_combined, width,
            color='#00FFFF')

ax.set_title('Model Accuracy on "Opinion Users"')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('NB', 'SVM', 'RF', 'DecTree', 'kNN'))

ax.legend((p1[0], p2[0], p3[0]), ('Non-Text Data', 'Text Data', 'Combined'), prop={'size': 6})
ax.set_ylim(0,100)
ax.autoscale_view()

plt.ylabel('Prediction Accuracy (%)')
plt.show()

In [ ]:
def make_bar_chart(non_tuple, text_tuple, combined_tuple, metric_string, user_type):
    Num_Models = 5

    fig, ax = plt.subplots()

    ind = np.arange(Num_Models)    # the x locations for the groups
    width = 0.25         # the width of the bars
    p1 = ax.bar(ind, non_tuple, width, color='b')

    p2 = ax.bar(ind + width, text_tuple, width,
                color='g')

    p3 = ax.bar(ind + width + width, combined_tuple, width,
                color='#00FFFF')

    ax.set_title('Model ' + metric_string + ' on "' + user_type + ' Users"')
    ax.set_xticks(ind + width / 2)
    ax.set_xticklabels(('NB', 'SVM', 'RF', 'DecTree', 'kNN'))

    ax.legend((p1[0], p2[0], p3[0]), ('Non-Text Data', 'Text Data', 'Combined'), prop={'size': 6})
    ax.set_ylim(0, 100)
    ax.autoscale_view()

    plt.ylabel('Prediction ' + metric_string +  '(%)')
    plt.show()
    

In [ ]:
make_bar_chart(mem_no_text_precision, mem_text_precision, mem_combined_precision, "Precision", "Memory")

In [ ]:
make_bar_chart(tech_no_text_precision, tech_text_precision, tech_combined_precision, "Precision", "Opinion")

In [ ]:
make_bar_chart(mem_no_text_recall, mem_text_recall, mem_combined_recall, "Recall", "Memory")

In [ ]:
make_bar_chart(tech_no_text_recall, tech_text_recall, tech_combined_recall, "Recall", "Opinion")

In [ ]:
make_bar_chart(mem_no_text_f1, mem_text_f1, mem_combined_f1, "F1", "Memory")

In [ ]:
make_bar_chart(tech_no_text_f1, tech_text_f1, tech_combined_f1, "F1", "Opinion")